<a href="https://colab.research.google.com/github/kobemensah001/CV_PROJECT_AAI_521_001/blob/main/Face_mask_project_ahmed.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
annotations_path = '/content/drive/MyDrive/Face mask dataset/annotations'
images_path = '/content/drive/MyDrive/Face mask dataset/images'



In [ ]:
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import os
import matplotlib.pyplot as plt
import matplotlib.patches as patches

In [ ]:
import glob
import xml.etree.ElementTree as ET  # Import for parsing XML files

annotations_path = '/content/drive/MyDrive/Face mask dataset/annotations'  # Your annotations path

# Initialize the dataset dictionary
dataset = {
    "file":[],
    "name":[],
    "width":[],
    "height":[],
    "xmin":[],
    "ymin":[],
    "xmax":[],
    "ymax":[],
}

# Iterate through each XML file in the annotations directory
for anno in glob.glob(annotations_path + "/*.xml"):
    tree = ET.parse(anno)
    width, height = None, None  # Initialize width and height

    for elem in tree.iter():
        if 'size' in elem.tag:
            for attr in list(elem):
                if 'width' in attr.tag:
                    width = int(round(float(attr.text)))
                if 'height' in attr.tag:
                    height = int(round(float(attr.text)))

        if 'object' in elem.tag:
            for attr in list(elem):
                if 'name' in attr.tag:
                    name = attr.text
                    dataset['name'].append(name)
                    dataset['width'].append(width)
                    dataset['height'].append(height)
                    dataset['file'].append(anno.split('/')[-1][:-4])

                if 'bndbox' in attr.tag:
                    for dim in list(attr):
                        if 'xmin' in dim.tag:
                            xmin = int(round(float(dim.text)))
                            dataset['xmin'].append(xmin)
                        if 'ymin' in dim.tag:
                            ymin = int(round(float(dim.text)))
                            dataset['ymin'].append(ymin)
                        if 'xmax' in dim.tag:
                            xmax = int(round(float(dim.text)))
                            dataset['xmax'].append(xmax)
                        if 'ymax' in dim.tag:
                            ymax = int(round(float(dim.text)))
                            dataset['ymax'].append(ymax)


In [ ]:
df=pd.DataFrame(dataset)
df.head()

In [ ]:
# Function to parse an XML file and extract annotations
def parse_xml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    annotations = []

    for member in root.findall('object'):
        label = member.find('name').text
        bndbox = member.find('bndbox')
        xmin = int(bndbox.find('xmin').text)
        ymin = int(bndbox.find('ymin').text)
        xmax = int(bndbox.find('xmax').text)
        ymax = int(bndbox.find('ymax').text)
        annotations.append({'label': label, 'bbox': [xmin, ymin, xmax, ymax]})

    filename = root.find('filename').text
    return filename, annotations

In [ ]:
# Function to preprocess a single image and its annotations for a fixed size of 300x300
def preprocess_image_annotation(image_path, annotation_list, target_size=(300, 300)):
    # Load the image
    image = Image.open(image_path)
    image = image.convert('RGB')  # Ensure image is in RGB format

    # Original size
    original_size = image.size

    # Resize image to 600x600
    image = image.resize(target_size)

    # Normalize the pixel values to be between 0 and 1
    image_array = np.array(image) / 255.0

    # Scale the bounding box annotations
    scale_x = target_size[0] / original_size[0]
    scale_y = target_size[1] / original_size[1]
    scaled_annotations = []
    for ann in annotation_list:
        scaled_bbox = [int(ann['bbox'][0] * scale_x), int(ann['bbox'][1] * scale_y),
                       int(ann['bbox'][2] * scale_x), int(ann['bbox'][3] * scale_y)]
        scaled_annotations.append({'label': ann['label'], 'bbox': scaled_bbox})

    return image_array, scaled_annotations

In [ ]:
annotations_dir = r'/content/drive/MyDrive/Face mask dataset/annotations'
images_dir = r'/content/drive/MyDrive/Face mask dataset/images'


In [ ]:
# List all XML files in the annotations directory
xml_files = [f for f in os.listdir(annotations_dir) if f.endswith('.xml')]

In [ ]:
# Preprocess all images and annotations
preprocessed_data = []
for xml_file in xml_files:
    # Parse annotations
    filename, annotations = parse_xml(os.path.join(annotations_dir, xml_file))

    # Corresponding image path
    image_filename = filename.replace('.xml', '.png')  # Ensure the extension matches your image files
    image_path = os.path.join(images_dir, image_filename)

    # Preprocess image and annotations
    image_array, scaled_annotations = preprocess_image_annotation(image_path, annotations)

    # Store in the list
    preprocessed_data.append({'image': image_array, 'annotations': scaled_annotations})

In [ ]:
# Visualize one example
sample_image_data = preprocessed_data[0]['image']
sample_annotations = preprocessed_data[0]['annotations']
plt.imshow(sample_image_data)
for ann in sample_annotations:
    bbox = ann['bbox']
    rect = patches.Rectangle((bbox[0], bbox[1]), bbox[2] - bbox[0], bbox[3] - bbox[1],
                             linewidth=1, edgecolor='r' if ann['label'] == 'without_mask' else 'g',
                             facecolor='none')
    plt.gca().add_patch(rect)
    plt.text(bbox[0], bbox[1], ann['label'], color='yellow', verticalalignment='top',
             bbox={'color': 'black', 'pad': 0})
plt.show()

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, auc

# Convert annotations to a suitable format for model training
labels = [1 if 'without_mask' in [ann['label'] for ann in data['annotations']] else 0 for data in preprocessed_data]

# Prepare features and labels for the model
features = np.array([data['image'] for data in preprocessed_data])
labels = np.array(labels)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Image augmentation
train_datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# Load a pre-trained VGG16 model as a base
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(300, 300, 3))
base_model.trainable = False  # Freeze the base model layers

# Create the model
model = Sequential([
    base_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Use 'softmax' for multiclass classification
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

# Fit the model
history = model.fit(
    train_datagen.flow(X_train, y_train, batch_size=16),
    validation_data=(X_test, y_test),
    epochs=10
)

# Plotting training and validation loss and accuracy
plt.figure(figsize=(12, 5))
epochs = range(1, 11)

# Training and Validation Loss
plt.subplot(1, 2, 1)
plt.plot(epochs, history.history['loss'], label='Training Loss')
plt.plot(epochs, history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Training and Validation Accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, history.history['accuracy'], label='Training Accuracy')
plt.plot(epochs, history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy}")

# Generate predictions and calculate additional metrics
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).astype(int)

# Calculate metrics
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Precision: {precision}, Recall: {recall}, F1-Score: {f1}")

# Generate prediction probabilities for ROC curve
y_pred_prob = model.predict(X_test).ravel()

# Compute ROC curve and ROC area
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plotting the ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:0.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

# Metrics
metrics = {
    "Metric": ["Accuracy", "Precision", "Recall", "F1-Score", "ROC AUC"],
    "Value": [test_accuracy, precision, recall, f1, roc_auc]
}

# Create DataFrame
metrics_df = pd.DataFrame(metrics)

# Display the DataFrame
print(metrics_df)


Analysis:
Training and Validation Accuracy:
The training accuracy increased from 60.85% to 72.58% over 10 epochs, which is a good sign of learning.
The validation accuracy also improved, reaching 75.44% in the final epoch.
However, the gap between training and validation accuracy suggests some overfitting.
Loss:
Both training and validation loss decreased over epochs, indicating that the model is learning effectively.
The fluctuations in validation loss suggest that the model might benefit from further tuning.
Precision, Recall, and F1-Score:
Precision is high (75%), indicating that when the model predicts a positive class, it is correct 75% of the time.
Recall is low (28.85%), meaning the model is missing a significant number of actual positive cases.
The F1-score, which balances precision and recall, is relatively low (41.67%). This indicates that the model is not effectively capturing the positive class.
ROC AUC:
The ROC AUC score is 76.57%, which is decent but indicates room for improvement, especially in distinguishing between the classes.
Suggestions for Improvement:
Address Overfitting:
Consider adding more Dropout layers or increasing the dropout rate.
Implement data augmentation if not already done.
Experiment with regularization techniques like L1 or L2 regularization.
Improve Recall:
Since recall is low, consider adjusting the class weights to give more importance to the positive class.
Experiment with different decision thresholds, instead of the default 0.5, to improve recall.
Model Architecture and Hyperparameters:
Try different model architectures or adjust the existing one.
Experiment with different hyperparameters, like the number of neurons in Dense layers or learning rate.
Cross-validation:
Use k-fold cross-validation to ensure that the model's performance is consistent across different subsets of the training data.
Additional Training:
More training epochs might help, but be cautious of overfitting.
Consider unfreezing some of the layers of the base model for fine-tuning.
By implementing these suggestions, you should be able to improve the model's ability to generalize to new data, as well as its capacity to correctly identify more positive cases (thus improving recall and F1-score).

Adjusting the Learning Rate

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Assuming 'model' is your Keras model and 'X_train', 'y_train' are your training data

# Define the optimizer with a higher or lower learning rate
optimizer = Adam(learning_rate=0.001)  # Adjust the learning rate as needed

# Compile the model with the new optimizer
model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])

# Reduce learning rate when a metric has stopped improving
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.0001)

# Fit the model
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2, callbacks=[reduce_lr])

# Now you can evaluate, make predictions, or plot the training history


Based on the modification you've made to your model, it seems that you have altered the learning rate of the optimizer and added a callback to reduce the learning rate when the validation loss stops improving. These are common techniques to improve model performance, particularly in addressing issues like overfitting or helping the model to converge better.

Given these modifications, here's how you can compare the performance of your model before and after these changes:

Training and Evaluating the Original Model:
Train your original model (without the learning rate adjustments and ReduceLROnPlateau callback).
Evaluate the model on your test data and store the predictions.
Optionally, save the trained model or its weights for future use.
Applying Modifications and Retraining:
Apply the modifications (as per your provided code).
Retrain the model on the same training data.
Evaluate the modified model on your test data and store these new predictions.
Plot ROC Curves:
Use the stored predictions from both model states to plot the ROC curves for comparison.
Assuming you've done the above steps, here's how you can plot the ROC curves:

python


 loss, accuracy, and the ROC curve. The ROC curve, in particular, provides a comprehensive view of the trade-off between the true positive rate and false positive rate across different thresholds, making it a useful tool for evaluating model performance.

Performance Metrics Before Modification:
Initial Training Loss: 0.6860
Final Training Loss: 0.5248
Initial Training Accuracy: 62.90%
Final Training Accuracy: 72.87%
Validation Loss and Accuracy showed fluctuations.
Performance Metrics After Modification:
Initial Training Loss: 0.6597
Final Training Loss: 0.1633
Initial Training Accuracy: 68.62%
Final Training Accuracy: 92.11%
Validation Loss started lower but increased slightly, suggesting potential overfitting.
Analysis:
Training Performance: The training loss decreased significantly, and the training accuracy increased notably after the modification. This indicates better learning and model fitting to the training data.
Validation Performance: While the initial validation loss was lower, it did not decrease consistently, and there was a slight increase in later epochs. This might suggest overfitting, as the model is performing increasingly better on the training data but not as well on the validation data.

Step 1: Modify the Model Architecture
You can add Dropout layers to your existing model structure. Let's assume your model is a Sequential model with layers defined previously. The following is a template to add Dropout:

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import precision_score, recall_score, f1_score, roc_curve, auc
import pandas as pd

# Example preprocessed_data loading (you should load your actual data here)
# preprocessed_data = ...

# Convert annotations to a suitable format for model training
labels = [1 if 'without_mask' in [ann['label'] for ann in data['annotations']] else 0 for data in preprocessed_data]

# Prepare features and labels for the model
features = np.array([data['image'] for data in preprocessed_data])
labels = np.array(labels)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Image augmentation
train_datagen = ImageDataGenerator(
    rotation_range=20,
    zoom_range=0.15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# Load a pre-trained VGG16 model as a base
base_model = VGG16(weights="imagenet", include_top=False, input_shape=(300, 300, 3))
base_model.trainable = False  # Freeze the base model layers

# Create the model
model = Sequential([
    base_model,
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Use 'softmax' for multiclass classification
])

# Compile the model
model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.0001), metrics=['accuracy'])

# Fit the model
history = model.fit(
    train_datagen.flow(X_train, y_train, batch_size=16),
    validation_data=(X_test, y_test),
    epochs=10
)

# Plotting training and validation loss and accuracy
plt.figure(figsize=(12, 5))
epochs = range(1, 11)

# Training and Validation Loss
plt.subplot(1, 2, 1)
plt.plot(epochs, history.history['loss'], label='Training Loss')
plt.plot(epochs, history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

# Training and Validation Accuracy
plt.subplot(1, 2, 2)
plt.plot(epochs, history.history['accuracy'], label='Training Accuracy')
plt.plot(epochs, history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()

plt.tight_layout()
plt.show()

# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print(f"Test accuracy: {test_accuracy}")

# Generate predictions and calculate additional metrics
y_pred = model.predict(X_test)
y_pred = np.round(y_pred).astype(int)

# Calculate metrics
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print(f"Precision: {precision}, Recall: {recall}, F1-Score: {f1}")

# Generate prediction probabilities for ROC curve
y_pred_prob = model.predict(X_test).ravel()

# Compute ROC curve and ROC area
fpr, tpr, _ = roc_curve(y_test, y_pred_prob)
roc_auc = auc(fpr, tpr)

# Plotting the ROC curve
plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:0.2f})')
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

# Metrics Table
metrics = {
    "Metric": ["Accuracy", "Precision", "Recall", "F1-Score", "ROC AUC"],
    "Value": [test_accuracy, precision, recall, f1, roc_auc]
}

# Create DataFrame
metrics_df


Training and Validation Accuracy:
The training accuracy improved from 62.61% to 69.79% over 10 epochs.
The validation accuracy reached 71.93% by the final epoch.
The model shows reasonable learning, but there's a noticeable gap between training and validation accuracy, indicating some degree of overfitting.
Loss:
Both training and validation loss decreased, with some fluctuations in validation loss. This is typical in training and suggests that the model might benefit from further tuning or regularization.
Precision, Recall, and F1-Score:
Precision is moderate (53.13%), indicating that when the model predicts a positive class, it is correct about half of the time.
Recall is higher (65.38%), meaning the model is better at identifying actual positive cases.
The F1-score, which balances precision and recall, is 58.62%. This score is decent but could be improved.
ROC AUC:
The ROC AUC score is not provided for the VGG16 model.

Performance Comparison Table:
Metric	VGG16 Model	Previous Model
Accuracy	71.93%	75.44%
Precision	53.13%	75.00%
Recall	65.38%	28.85%
F1-Score	58.62%	41.67%
ROC AUC	N/A	76.57%
Observations and Suggestions:

In [ ]:
Performance Comparison Table:
Metric	VGG16 Model	Previous Model
Accuracy	71.93%	75.44%
Precision	53.13%	75.00%
Recall	65.38%	28.85%
F1-Score	58.62%	41.67%
ROC AUC	N/A	76.57%


Observations and Suggestions:
Overall Performance: The VGG16 model has a higher recall but lower precision and accuracy compared to the previous model. This suggests that VGG16 is better at identifying positive cases but also makes more false positive errors.
Overfitting: There is evidence of overfitting in the VGG16 model, as seen by the gap between training and validation accuracy.
F1-Score: The F1-score is higher in the VGG16 model, indicating a better balance between precision and recall compared to the previous model.
Suggestions for Improvement:
Regularization: To address overfitting, consider adding more Dropout layers or increasing the dropout rate. Regularization techniques like L1 or L2 regularization can also be beneficial.
Data Augmentation: More aggressive data augmentation can help the model generalize better.
Model Tuning: Experiment with different architectures or hyperparameters, such as the number of neurons in Dense layers or learning rate adjustments.
Fine-Tuning: Unfreezing some of the layers in the pre-trained base model and training them along with the top layers might improve performance.
By adjusting these aspects, the VGG16 model's ability to generalize and its overall performance might be enhanced.